## Import Packages

In [14]:
import tensorflow as tf
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


## Read File

In [15]:
data = pd.read_csv('OnlineNewsPopularity.csv')
data.columns = [col.strip() for col in data.columns]


## Create isolation forest model to remove outliers
[API Link to IF](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)

In [16]:
old_data_length = len(data)
IF_model = IsolationForest(contamination=0.1) # Remove 10% data
outliers = IF_model.fit_predict(data.drop(columns=['url', 'timedelta', 'shares'])) # Delete columns than don't need in IF and train IF model
data['outliers'] = outliers

# Remove outliers
data_outliers = data[data['outliers'] == -1]
new_data = data[data['outliers'] != -1].drop(columns = ['outliers'])

new_data_len = len(new_data)

print(f"There are total {old_data_length - new_data_len} rows been removied.")


There are total 3965 rows been removied.


## Creat a function to classify an article to different categories
* Great Article: Shares >= 90%
* Good Article: 70% <= Shares < 90%
* Normal Article: 30% <= Shares < 70%
* Bad article: Shares < 30%

In [17]:
def article_classifier(shares, top_percent):
    if shares >= top_percent[0.9]:
        return 'Great'
    elif shares >= top_percent[0.7]:
        return 'Good'
    elif shares >= top_percent[0.3]:
        return 'Normal'
    else:
        return 'Bad'
        

## Data Classifier

In [18]:
top_percent = new_data['shares'].quantile([0.3, 0.7, 0.9])

quality = []
for index, row in new_data.iterrows():
    quality.append(article_classifier(row['shares'], top_percent))

new_data['quality'] = quality


## Split dataset into x and y and one-hot encode quality column

In [19]:
x = new_data.drop(columns=['quality', 'url', 'timedelta', 'shares'], axis=1)
y = OneHotEncoder().fit_transform(new_data[['quality']]).toarray()


## Train test split

In [20]:

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=10)


## Scale the data to Improve the speed of training SVM Linear Model

In [21]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
scale_xTrain = standard_scaler.fit_transform(xTrain)
scale_xTest = standard_scaler.transform(xTest)

## Create and train model(NN)

In [22]:
nn_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(scale_xTrain.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(
    scale_xTrain,
    yTrain,
    epochs=10
)

Epoch 1/10
892/892 [==============================] - 3s 3ms/step - loss: 1.2188 - accuracy: 0.4241
Epoch 2/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1805 - accuracy: 0.4508
Epoch 3/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1663 - accuracy: 0.4600
Epoch 4/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1557 - accuracy: 0.4666
Epoch 5/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1440 - accuracy: 0.4713
Epoch 6/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1319 - accuracy: 0.4792
Epoch 7/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1200 - accuracy: 0.4874
Epoch 8/10
892/892 [==============================] - 3s 3ms/step - loss: 1.1072 - accuracy: 0.4952
Epoch 9/10
892/892 [==============================] - 3s 3ms/step - loss: 1.0922 - accuracy: 0.5043
Epoch 10/10
892/892 [==============================] - 3s 3ms/step - loss: 1.0796 - accuracy: 0.5129

## Create and train model(Random Forest)

In [23]:
RF_model = RandomForestClassifier(n_estimators=100)
RF_model.fit(scale_xTrain, yTrain.argmax(axis=1))


RandomForestClassifier()

## Create and train model(Polynomial Regression)

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(3), LinearRegression())
poly_model.fit(scale_xTrain, yTrain.argmax(axis=1))

## Create ensemble model

In [ ]:
nn_pred = nn_model(scale_xTest)
RF_pred = tf.keras.utils.to_categorical(RF_model.predict(scale_xTest), num_classes=4)
poly_pred = tf.keras.utils.to_categorical(poly_model.predict(scale_xTest), num_classes=4)

poly_pred = tf.keras.utils.to_categorical(poly_pred, num_classes=4)

ensemble = (nn_pred + RF_pred + poly_pred) / 3

## Model Evaluation

In [ ]:
tmp = tf.argmax(ensemble, axis=1).numpy()
print(classification_report(yTest.argmax(axis=1), tmp))
print(f"Ensemble Model Accuracy: {accuracy_score(yTest.argmax(axis=1), tmp)}")